In [1]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    answer_similarity,
)

from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    answer_similarity,
)
from ragas import evaluate
import pandas as pd


c:\Users\SSAFY\Desktop\project_2\S12P21A702\finafanAI\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 평가할 메트릭 리스트
metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    answer_similarity
]


In [2]:
import asyncio
from app.service.search import fast_news_search
from app.service.prompts import idol_news_prompt
from langchain_openai import ChatOpenAI
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)
from datasets import Dataset  # ✅ 핵심
import pandas as pd
import matplotlib.pyplot as plt

# ✅ 질문 리스트
sample_questions = [
    "이찬원 요즘 뭐해?",
    "임영웅 요즘 뭐해?",
    "영웅이 소식 알려줘",
    "임영웅 뉴스 찾아줘",
    "정동원 뉴스 찾아",
    "박지민 최근 기사 있어?",
    "방탄소년단 관련 뉴스 보여줘",
    "아이유 뉴스 뭐 떴어?",
    "이찬원 관련 기사 찾아줘",
    "장원영 근황 기사 알려줘",
    "지민이 무슨 뉴스 있어?",
    "정국 기사 좀 줘",
    "수지 요즘 뭐하고 있어?",
    "임영웅 기사 뭐 나왔어?",
    "이찬원 근황 알려줘",
    "아이브 뉴스 보여줘",
    "엔하이픈 관련 뉴스 있을까?",
    "김세정 최근 기사 뭐 있어?",
    "에스파 뉴스 보여줘",
    "르세라핌 기사 좀 찾아줘"
]

# ✅ 체인 구성
llm = ChatOpenAI(temperature=0.1)
idol_news_chain = idol_news_prompt | llm

# ✅ 평가용 샘플 생성 함수
async def build_ragas_sample(question: str):
    try:
        docs = fast_news_search(question) or ["(관련 기사가 없습니다.)"]
        contexts = docs[:5]

        input_data = {
            "topic": question,
            "results": docs
        }

        response = await idol_news_chain.ainvoke(input_data)

        return {
            "question": question,
            "contexts": contexts if isinstance(contexts, list) else [str(contexts)],
            "answer": response.content.strip(),
            "ground_truth": None
        }

    except Exception as e:
        print(f"❌ [{question}] 처리 중 오류: {e}")
        return None


# ✅ 평가 루프
async def run_ragas_evaluation():
    print("🚀 뉴스 기반 체인 평가 시작...")
    samples = await asyncio.gather(*[build_ragas_sample(q) for q in sample_questions])
    samples = [s for s in samples if s]

    if not samples:
        print("⚠️ 유효한 샘플이 없습니다. 평가를 종료합니다.")
        return

    # ✅ HuggingFace Dataset으로 변환
    df = pd.DataFrame(samples)
    ragas_dataset = Dataset.from_pandas(df)

    print(f"\n✅ 총 {len(ragas_dataset)}개 샘플 평가 준비 완료")

    # ✅ 평가 실행
# ✅ 평가 실행 (ground_truth 없이 평가 가능한 메트릭만 사용)
    results = evaluate(
        ragas_dataset,
        metrics=[
        faithfulness,
        answer_relevancy
        # context_precision,
        # context_recall
        ]
    )


    print("\n📊 RAGAS 평가 결과:")
    print(results)

    # ✅ 그래프 출력
    results.plot(kind='bar', figsize=(10, 5), title="RAGAS Evaluation Metrics", ylim=(0, 1))
    plt.ylabel("Score")
    plt.xticks(rotation=45)
    plt.grid(axis="y", linestyle="--")
    plt.tight_layout()
    plt.show()

    # ✅ CSV 저장
    df.to_csv("idol_news_ragas_dataset.csv", index=False)
    results.to_csv("idol_news_ragas_metrics.csv")

# ✅ 실행
if __name__ == "__main__":
    asyncio.run(run_ragas_evaluation())


ModuleNotFoundError: No module named 'app'

In [ ]:
asyncio.run(run_ragas_evaluation())

In [1]:
import asyncio
import pandas as pd
from urllib.parse import quote
import feedparser
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
load_dotenv()

# ✅ GPT-4 LLM
llm = ChatOpenAI(model="gpt-4", temperature=0.1)

# ✅ 뉴스 검색 함수
def fast_news_search(query: str) -> list[str]:
    encoded_query = quote(query)
    url = f"https://news.google.com/rss/search?q={encoded_query}&hl=ko&gl=KR&ceid=KR:ko"

    feed = feedparser.parse(url)
    results = []
    for entry in feed.entries[:3]:  # 상위 3개
        title = entry.title
        link = entry.link
        results.append(f"{title} - {link}")
    return results

# ✅ 요약 프롬프트
gt_prompt = PromptTemplate.from_template("""
다음은 "{topic}"에 대한 뉴스 제목 목록입니다.
이 제목들을 바탕으로 시니어도 쉽게 이해할 수 있도록 간단하고 정확한 요약을 작성해줘.
문장은 2~3문장 정도로 해줘.

뉴스 제목 목록:
{titles}

요약:
""")

# ✅ 체인 구성
generate_gt_chain = gt_prompt | llm

# ✅ Ground Truth 생성 함수
async def generate_ground_truth(question: str, titles: list[str]) -> str:
    input_data = {
        "topic": question,
        "titles": "\n".join(titles)
    }
    response = await generate_gt_chain.ainvoke(input_data)
    return response.content.strip()

async def build_gt_sample(question: str):
    try:
        titles = fast_news_search(question)

        if not titles or "(관련 기사가 없습니다" in titles[0]:
            return None

        ground_truth = await generate_ground_truth(question, titles)

        print(f"\n📌 질문: {question}")
        print("📰 뉴스 제목:")
        for t in titles:
            print(f"  - {t}")
        print(f"✅ 요약문(GT): {ground_truth}")
        print("-" * 60)

        return {
            "question": question,
            "news_titles": "\n".join(titles),
            "ground_truth": ground_truth
        }

    except Exception as e:
        print(f"❌ [{question}] 처리 중 오류: {e}")
        return None

# ✅ 전체 실행 루프
async def generate_gt_dataset(sample_questions: list[str]):
    print("🚀 Ground Truth 생성 시작...\n")
    samples = await asyncio.gather(*[build_gt_sample(q) for q in sample_questions])
    samples = [s for s in samples if s]

    df = pd.DataFrame(samples)
    df.to_excel("generated_gt_dataset.xlsx", index=False)
    print("\n📁 CSV 저장 완료: generated_gt_dataset.csv")




In [2]:
sample_questions_extended = [
    "이찬원 요즘 뭐해?",
    "임영웅 요즘 뭐해?",
    "영웅이 소식 알려줘",
    "임영웅 뉴스 찾아줘",
    "정동원 뉴스 찾아",
    "박지민 최근 기사 있어?",
    "방탄소년단 관련 뉴스 보여줘",
    "아이유 뉴스 뭐 떴어?",
    "이찬원 관련 기사 찾아줘",
    "장원영 근황 기사 알려줘",
    "지민이 무슨 뉴스 있어?",
    "정국 기사 좀 줘",
    "수지 요즘 뭐하고 있어?",
    "임영웅 기사 뭐 나왔어?",
    "이찬원 근황 알려줘",
    "아이브 뉴스 보여줘",
    "엔하이픈 관련 뉴스 있을까?",
    "김세정 최근 기사 뭐 있어?",
    "에스파 뉴스 보여줘",
    "르세라핌 기사 좀 찾아줘",
    "이무진 최근 기사 좀",
    "장윤정 무슨 기사 있어?",
    "비 뉴스 뭐 나왔어?",
    "선미 소식 알려줘",
    "제니 뉴스 좀 줘",
    "지드래곤 기사 있어?",
    "태연 관련 뉴스 없어?",
    "정은지 근황 어때?",
    "이서 뉴스 뭐 있어?",
    "이효리 요즘 기사 떴어?",
    "유재석 뭐라고 했어?",
    "조세호 뉴스 봤어?",
    "신동엽 기사 줘봐",
    "송중기 요즘 뭐해?",
    "박보검 기사 있니?",
    "박서준 관련 기사 뭐 있어?",
    "정해인 요즘 소식 있어?",
    "공유 뉴스 좀 알려줘",
    "서인국 기사 나왔어?",
    "윤아 최근 뉴스 뭐야?",
    "채원 뉴스 보여줘",
    "미연 기사 좀 줄래?",
    "한소희 관련 기사 알려줘",
    "문빈 뉴스 있니?",
    "이도현 기사 줘",
    "남주혁 최근 뉴스 뭐 있어?",
    "박민영 뉴스 뭐 떴어?",
    "수현이 근황 어때?",
    "태양 최근 기사 봤어?",
    "현아 뉴스 좀 알려줘",
]


In [3]:
await generate_gt_dataset(sample_questions_extended)

🚀 Ground Truth 생성 시작...


📌 질문: 태양 최근 기사 봤어?
📰 뉴스 제목:
  - "이거 못봤어?" 여의도에서 가장 핫한 리포트 - 머니투데이방송MTN - https://news.google.com/rss/articles/CBMiZEFVX3lxTE9QS1kwLUVYMk5mdEUyWnppNTNhSU5FeGFCWlpCc25hLV94T3dZVHoxSTNSSU0yRFN1RE5aa01XSTVJOG9ra0Y3NlVEaUkwWllpSFpuMldudlFrM00xUFQ5aVliLTg?oc=5
✅ 요약문(GT): 여의도에서 가장 주목받는 리포트에 대한 뉴스가 있었습니다. 자세한 내용은 기사를 통해 확인하실 수 있습니다.
------------------------------------------------------------

📌 질문: 송중기 요즘 뭐해?
📰 뉴스 제목:
  - 송중기 최근 모습, 여전히 굴욕없는 비주얼 '눈길' - 네이트 뉴스 - https://news.google.com/rss/articles/CBMiYEFVX3lxTE5YbjR5Z3hEQ2VPMmlYOWFKTnBFOWstVzZibkFRaUtSdkxpVTdScS14NHNMcmh4U1pUQXhpUWtJN2QxaEQ2SGRoQzJpVTRGZWdTbG9iT0swdDE3OG9BWHVBSg?oc=5
✅ 요약문(GT): 배우 송중기의 최근 모습이 공개되어 눈길을 끌고 있다. 그의 비주얼은 여전히 굴욕 없이 뛰어나다.
------------------------------------------------------------

📌 질문: 비 뉴스 뭐 나왔어?
📰 뉴스 제목:
  - "나 뉴스 나왔어" 조롱 여전‥추가 피해자 속출 - MBC 뉴스 - https://news.google.com/rss/articles/CBMiekFVX3lxTE5kWHBlYTNDbTVuUC1hTmRrd0VhcHZ4OS1lekRLb2xjUjE4T0tFcjdvMmRrZi04NmE0UHpscFVvWF8tbHBFMmZPaDRf